## Title

#### Introduction

- Relevant background of topic: rewriting README file, simple down so outsider can understand dataset and meaning behind dataset.

- Question (word this better): Whether we are able to classify a star as a pulsar or not by doing classification analysis, determine accuracy prediction, and recall of classification analysis. 

- describe dataset: summary of README, explaining data meanings, etc. 


#### Preliminary exploratory data analysis

- Read in dataset
- clean and wrangle
- summarize table in one data
- visualize data with one plot relevant to our analysis

In [ ]:
# import dataset

In [ ]:
# clean

In [ ]:
# table - using only training data

In [ ]:
# plot - using only training data

#### Methods

Explain

Describe

#### Expected outcomes and Significance

- What we will find? Look at visualizations, data, etc.
- Impact of findings?
- Future questions? 